In [2]:
import numpy as np
import pandas as pd
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
df_=pd.read_excel(r'C:\GITHUB\RFM Analysis\Data\online.xlsx', engine='openpyxl')


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.